In [1]:
import os
import math
import numpy as np
import pandas as pd
import rasterio as rio
from rasterio.windows import from_bounds

In [2]:
# Set directory and file names
data_dir = r'/Users/yasuyukiakita/Data'
results_dir = r'/Users/yasuyukiakita/Projects/python_gis_py312/results'
landfire_dir = os.path.join(data_dir, 'landfire')
lf_2023_fccs_raster = os.path.join(landfire_dir, 'LF2023_FCCS_240_CONUS', 
                                   'Tif', 'LC23_FCCS_240.tif')

# Extract raster using envelope

## Compute adjusted bounds

In [3]:
# Envelope coordinates (longitude/latitude or projected units)
xmin = -2362390
xmax = -1575139
ymin = 1200000
ymax = 2472128

In [4]:
def compute_adjusted_bounds(src_raster, xmin, ymin, xmax, ymax):
    # Read source raster
    with rio.open(src_raster) as src:
        src_crs = src.crs
        src_pixel_x, src_pixel_y = src.res
        src_bounds = src.bounds

    # Check xmin, xmax, ymin, ymax
    if xmin < src_bounds[0]:
        raise ValueError('xmin is smaller than xmin of source extent')
    if xmax > src_bounds[2]:
        raise ValueError('xmax is larger than xmax of source extent')
    if ymin < src_bounds[1]:
        raise ValueError('ymin is smaller than ymin of source extent')
    if ymax > src_bounds[3]:
        raise ValueError('ymax is larger than ymax of source extent')

    # Compute adjusted bounds
    adj_xmin = math.floor((xmin - src_bounds[0]) / src_pixel_x) * src_pixel_x + src_bounds[0]
    adj_xmax = math.ceil((xmax - src_bounds[0]) / src_pixel_x) * src_pixel_x + src_bounds[0]
    adj_ymin = math.floor((ymin - src_bounds[1]) / src_pixel_y) * src_pixel_y + src_bounds[1]
    adj_ymax = math.ceil((ymax - src_bounds[1]) / src_pixel_y) * src_pixel_y + src_bounds[1]

    # Check adjusted xmin, xmax, ymin, ymax
    if adj_xmin > xmin:
        raise ValueError('adjusted xmin is larger than original xmin')
    if adj_xmax < xmax:
        raise ValueError('adjusted xmax is smaller than original xmax')
    if adj_ymin > ymin:
        raise ValueError('adjusted ymin is larger than original ymin')
    if adj_ymax < ymax:
        raise ValueError('adjusted ymax is smaller than original ymax')
    
    return [adj_xmin, adj_ymin, adj_xmax, adj_ymax]

In [5]:
compute_adjusted_bounds(lf_2023_fccs_raster, xmin, ymin, xmax, ymax)

[-2362395.0, 1199985.0, -1575135.0, 2472135.0]

## Function to extract raster by envelope

In [6]:
def extract_raster_by_envelope(src_raster, dst_raster, xmin, ymin, xmax, ymax):
    # Adjuste xmin, xmax, ymin, ymax
    adj_xmin, adj_ymin, adj_xmax, adj_ymax = compute_adjusted_bounds(src_raster, xmin, ymin, xmax, ymax)
    # Open the source raster
    with rio.open(src_raster) as src:
        # Create a window from the envelope bounds
        window = from_bounds(adj_xmin, adj_ymin, adj_xmax, adj_ymax, 
                             transform=src.transform)
        
        # Read the data within the window
        data = src.read(window=window)
        
        # Calculate the transform for the new window
        transform = src.window_transform(window)
        
        # Update metadata for output
        out_meta = src.meta.copy()
        out_meta.update({
            "driver": "GTiff",
            "height": window.height,
            "width": window.width,
            "transform": transform, 
            "compress": 'lzw', 
        })
        
        # Write the cropped raster to a new file
        with rio.open(dst_raster, "w", **out_meta) as dest:
            dest.write(data)    

In [7]:
dst_raster = os.path.join(results_dir, 'landfire', 'fccs_value_CA_only.tif')
extract_raster_by_envelope(lf_2023_fccs_raster, dst_raster, xmin, ymin, xmax, ymax)

## Convert raster value to FCCS ID

In [8]:
lf_csv_final_df = pd.read_csv(os.path.join(results_dir, 'landfire', 'raster_value_fccsid_table.csv'))

In [9]:
lf_csv_final_df

,VALUE,FCCSID_base,FCCS,FCCSID,FCCS_REG,FUELBED
0,-1111,-1111,-1111,Fill-Not Mapped,All,Fill-Not Mapped
1,-9999,-9999,-9999,Fill-NoData,All,Fill-NoData
2,0,0,0,0,All,Bare Ground
3,1,1,1,1,Pacific Northwest,Black cottonwood-Douglas-fir-quaking aspen forest
4,2,2,2,2,Pacific Northwest,Western hemlock-western redcedar-Douglas-fir f...
...,...,...,...,...,...,...
15433,12990122,1299,12990122,1299-122,All,Corn and soybean (double crop) field
15434,12990123,1299,12990123,1299-123,All,Corn and soybean (double crop) field
15435,12990131,1299,12990131,1299-131,All,Corn and soybean (double crop) field
15436,12990132,1299,12990132,1299-132,All,Corn and soybean (double crop) field


In [10]:
with rio.open(dst_raster) as src:
    # Read the FCCS value data
    fccs_value_data = src.read(1)
    fccs_value_meta = src.meta.copy()

In [11]:
fccs_value_data

array([[-9999, -9999, -9999, ...,    52,    53,    53],
       [-9999, -9999, -9999, ...,    52,    53,    53],
       [-9999, -9999, -9999, ...,    52,    52,    53],
       ...,
       [-9999, -9999, -9999, ...,   307,   307,   307],
       [-9999, -9999, -9999, ...,   307,   307,   307],
       [-9999, -9999, -9999, ...,   307,   307,   307]],
      shape=(42405, 26242), dtype=int32)

In [12]:
unique_vals, count = np.unique(fccs_value_data, return_counts=True)

In [13]:
fccs_vals_count_df = pd.DataFrame({'unique_vals':unique_vals, 'count':count})

In [14]:
lf_csv_final_df = lf_csv_final_df.loc[lf_csv_final_df['VALUE'].isin(
    fccs_vals_count_df['unique_vals'])].reset_index(drop=True)

In [15]:
lf_csv_final_df

,VALUE,FCCSID_base,FCCS,FCCSID,FCCS_REG,FUELBED
0,-9999,-9999,-9999,Fill-NoData,All,Fill-NoData
1,0,0,0,0,All,Bare Ground
2,2,2,2,2,Pacific Northwest,Western hemlock-western redcedar-Douglas-fir f...
3,6,6,6,6,Pacific Northwest,Oregon white oak-Douglas-fir forest
4,7,7,7,7,Pacific Northwest,Douglas-fir-sugar pine-tanoak forest
...,...,...,...,...,...,...
1886,12810122,1281,12810122,1281-122,All,"Pasture, hay, or alfalfa field - grazed or har..."
1887,12810123,1281,12810123,1281-123,All,"Pasture, hay, or alfalfa field - grazed or har..."
1888,12810131,1281,12810131,1281-131,All,"Pasture, hay, or alfalfa field - grazed or har..."
1889,12810132,1281,12810132,1281-132,All,"Pasture, hay, or alfalfa field - grazed or har..."


In [16]:
fccs_id_data = np.ones(fccs_value_data.shape, dtype='int') * -8888

In [17]:
for idx, row in lf_csv_final_df.iterrows():
    fccs_value = row['VALUE']
    fccs_id = row['FCCSID_base']
    print('{0}: Convert {1} to {2}'.format(idx, fccs_value, fccs_id))
    mask = fccs_value_data == fccs_value
    fccs_id_data[mask] = fccs_id

0: Convert -9999 to -9999
1: Convert 0 to 0
2: Convert 2 to 2
3: Convert 6 to 6
4: Convert 7 to 7
5: Convert 8 to 8
6: Convert 10 to 10
7: Convert 12 to 12
8: Convert 13 to 13
9: Convert 14 to 14
10: Convert 15 to 15
11: Convert 16 to 16
12: Convert 20 to 20
13: Convert 22 to 22
14: Convert 24 to 24
15: Convert 28 to 28
16: Convert 30 to 30
17: Convert 32 to 32
18: Convert 36 to 36
19: Convert 37 to 37
20: Convert 39 to 39
21: Convert 40 to 40
22: Convert 41 to 41
23: Convert 42 to 42
24: Convert 43 to 43
25: Convert 44 to 44
26: Convert 45 to 45
27: Convert 46 to 46
28: Convert 47 to 47
29: Convert 48 to 48
30: Convert 49 to 49
31: Convert 51 to 51
32: Convert 52 to 52
33: Convert 53 to 53
34: Convert 56 to 56
35: Convert 57 to 57
36: Convert 59 to 59
37: Convert 60 to 60
38: Convert 61 to 61
39: Convert 62 to 62
40: Convert 65 to 65
41: Convert 67 to 67
42: Convert 69 to 69
43: Convert 77 to 77
44: Convert 210 to 210
45: Convert 213 to 213
46: Convert 214 to 214
47: Convert 215 to 21

In [18]:
unique_vals, count = np.unique(fccs_id_data, return_counts=True)
fccs_id_count_df = pd.DataFrame({'unique_vals':unique_vals, 'count':count})

In [19]:
fccs_id_count_df

,unique_vals,count
0,-9999,194483618
1,0,138350952
2,2,4716
3,6,725423
4,7,527964
...,...,...
114,1244,17869140
115,1247,87079
116,1261,7302561
117,1273,19412993


In [20]:
fccs_value_meta

{'driver': 'GTiff',
 'dtype': 'int32',
 'nodata': 2147483647.0,
 'width': 26242,
 'height': 42405,
 'count': 1,
 'crs': CRS.from_wkt('PROJCS["NAD83 / Conus Albers",GEOGCS["NAD83",DATUM["North_American_Datum_1983",SPHEROID["GRS 1980",6378137,298.257222101,AUTHORITY["EPSG","7019"]],AUTHORITY["EPSG","6269"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4269"]],PROJECTION["Albers_Conic_Equal_Area"],PARAMETER["latitude_of_center",23],PARAMETER["longitude_of_center",-96],PARAMETER["standard_parallel_1",29.5],PARAMETER["standard_parallel_2",45.5],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","5070"]]'),
 'transform': Affine(30.0, 0.0, -2362395.0,
        0.0, -30.0, 2472135.0)}

In [21]:
fccs_value_meta.update({
    "compress": 'lzw', 
})

In [24]:
# Write FCCS ID raster
fccs_id_raster = os.path.join(results_dir, 'landfire', 'fccs_id_CA_only.tif')
with rio.open(fccs_id_raster, "w", **fccs_value_meta) as dest:
    dest.write(fccs_id_data, 1)